In [9]:
import numpy as np
import matplotlib.pyplot as plt
import VisualDataMaker
import CNNRegressionAndSoftmax as CRS
import tensorflow as tf

In [10]:
data, OHE_LABELS = VisualDataMaker.combine("./random_BG_IMG/n_data/*.jpg", "./original_DOB_images/*.png")
#bounded_DOB = VisualDataMaker.plot_bounding_box(data)

In [3]:
batch = []
regr_truth = []
class_truth = []
for i in range(len(data)):
    batch.append(data[i][0])
    regr_truth.append([data[i][2][0], data[i][2][1], data[i][3]])
    class_truth.append(OHE_LABELS[data[i][1]])

In [4]:
batch = np.array(batch)
regr_truth = np.array(regr_truth)
class_truth = np.array(class_truth)

In [5]:
batch.shape, regr_truth.shape, class_truth.shape

((37450, 224, 224), (37450, 3), (37450, 10))

In [6]:
batch = tf.keras.backend.constant(batch)

In [ ]:
model = CRS.getUntrainedModel()

In [ ]:
model.summary()

In [ ]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="class_loss", patience=5, restore_best_weights=True),
    tf.keras.callbacks.EarlyStopping(monitor="Coordinates_Surface_Area_root_mean_squared_error", patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="Coordinates_Surface_Area_root_mean_squared_error", patience=2),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="class_loss", patience=2),
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]

'\nmy_callbacks = [\n    tf.keras.callbacks.EarlyStopping(monitor="class_accuracy", patience=5, restore_best_weights=True),\n    tf.keras.callbacks.EarlyStopping(monitor="boundary_box_mse", patience=5, restore_best_weights=True),\n    tf.keras.callbacks.ReduceLROnPlateau(monitor="class_accuracy", patience=2),\n    tf.keras.callbacks.ReduceLROnPlateau(monitor="boundary_box_mse", patience=2),\n    tf.keras.callbacks.TensorBoard(log_dir=\'./logs\')\n]\n'

In [ ]:
model.fit(batch, [class_truth, regr_truth], epochs=200, batch_size=64, callbacks=my_callbacks)

In [ ]:
model.save("./Saved_Models/CNN_Regression_and_Softmax")

In [ ]:
model = tf.keras.models.load_model("./saved_models/content/Saved_Models/CNN_Regression_and_Softmax/")